<a href="https://colab.research.google.com/github/mannmoshe/text-recognition/blob/main/text_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# based on:
# https://deeplearningcourses.com/c/data-science-natural-language-processing-in-python 
# https://www.udemy.com/data-science-natural-language-processing-in-python Section 7 Cipher Decryption

# Author: Moshe Mann

import numpy as np
np.seterr(divide = 'ignore') 

import string
import random
import requests
import pandas as pd

In [2]:
for l in range(1488, 1515):
  print(l, l - 1488, chr(l))

1488 0 א
1489 1 ב
1490 2 ג
1491 3 ד
1492 4 ה
1493 5 ו
1494 6 ז
1495 7 ח
1496 8 ט
1497 9 י
1498 10 ך
1499 11 כ
1500 12 ל
1501 13 ם
1502 14 מ
1503 15 ן
1504 16 נ
1505 17 ס
1506 18 ע
1507 19 ף
1508 20 פ
1509 21 ץ
1510 22 צ
1511 23 ק
1512 24 ר
1513 25 ש
1514 26 ת


In [3]:
### the language model

# initialize Markov matrix
# MM: matrix for the bi-gram probabilites
M = np.ones((22, 22))

# initial state distribution
# MM: vector for the uni-gram probabilites
# MM: uni-gram probabilites calculated only by first letters of the words
pi = np.zeros(22)

# a function to update the Markov matrix
def update_transition(ch1, ch2):
  # ord('א') = 1488, ord('ב') = 1489, ...
  if ord(ch1) <= 1497:
    i = ord(ch1) - 1488 # 'א' will mapped to 0, 'ב' will mapped to 1 and so on
  elif ord(ch1) <= 1500:
    i = ord(ch1) - 1489
  elif ord(ch1) <= 1502:
    i = ord(ch1) - 1490
  elif ord(ch1) <= 1506:
    i = ord(ch1) - 1491
  elif ord(ch1) <= 1508:
    i = ord(ch1) - 1492
  else:
    i = ord(ch1) - 1493

  if ord(ch2) <= 1497:
    j = ord(ch2) - 1488 
  elif ord(ch2) <= 1500:
    j = ord(ch2) - 1489
  elif ord(ch2) <= 1502:
    j = ord(ch2) - 1490
  elif ord(ch2) <= 1506:
    j = ord(ch2) - 1491
  elif ord(ch2) <= 1508:
    j = ord(ch2) - 1492
  else:
    j = ord(ch2) - 1493

  M[i,j] += 1

# a function to update the initial state distribution
def update_pi(ch):
  if ord(ch) <= 1497:
    i = ord(ch) - 1488 
  elif ord(ch) <= 1500:
    i = ord(ch) - 1489
  elif ord(ch) <= 1502:
    i = ord(ch) - 1490
  elif ord(ch) <= 1506:
    i = ord(ch) - 1491
  elif ord(ch) <= 1508:
    i = ord(ch) - 1492
  else:
    i = ord(ch) - 1493

  pi[i] += 1

# get the log-probability of a word / token
def get_word_prob(word):
  # print("word:", word)
  if ord(word[0]) <= 1497:
    i = ord(word[0]) - 1488 
  elif ord(word[0]) <= 1500:
    i = ord(word[0]) - 1489
  elif ord(word[0]) <= 1502:
    i = ord(word[0]) - 1490
  elif ord(word[0]) <= 1506:
    i = ord(word[0]) - 1491
  elif ord(word[0]) <= 1508:
    i = ord(word[0]) - 1492
  else:
    i = ord(word[0]) - 1493

  logp = np.log(pi[i])

  for ch in word[1:]:
    if ord(ch) <= 1497:
      j = ord(ch) - 1488 
    elif ord(ch) <= 1500:
      j = ord(ch) - 1489
    elif ord(ch) <= 1502:
      j = ord(ch) - 1490
    elif ord(ch) <= 1506:
      j = ord(ch) - 1491
    elif ord(ch) <= 1508:
      j = ord(ch) - 1492
    else:
      j = ord(ch) - 1493
    logp += np.log(M[i, j]) # update prob
    i = j # update i to j

  return logp

# get the probability of a sequence of words
def get_sequence_prob(words):
  # if input is a string, split into an array of tokens
  if type(words) == str:
    words = words.split()

  logp = 0
  for word in words:
    logp += get_word_prob(word)
  return logp

In [4]:
# with open('torah_text.txt', encoding='ISO-8859-8') as f:
#     torah_text = f.read()
req = requests.get("https://raw.githubusercontent.com/mannmoshe/text-recognition/main/torah_heb.txt")
req.encoding = 'ISO-8859-8'
torah_text = req.text

In [5]:
torah_text[:50]

'בראשית ברא אלהים את השמים ואת הארץ והארץ היתה תהו '

In [6]:
torah_text = torah_text.replace('ם','מ').replace('ן','נ').replace('ץ','צ').replace('ף','פ').replace('ך','כ')

In [7]:
torah_text[:50]

'בראשית ברא אלהימ את השמימ ואת הארצ והארצ היתה תהו '

In [8]:
torah_text_no_spaces = torah_text.replace(' ', '')

In [9]:
torah_text_no_spaces[:50]

'בראשיתבראאלהימאתהשמימואתהארצוהארצהיתהתהוובהווחשכעל'

In [10]:
len(torah_text_no_spaces)

304805

In [11]:
tokens = torah_text.split()

for token in tokens:
  # update the model

  # first letter
  ch0 = token[0]
  update_pi(ch0)

  # other letters
  for ch1 in token[1:]:
    update_transition(ch0, ch1)
    ch0 = ch1

pi_initial_count = pi.copy()
M_initial_count = M.copy()

# normalize the probabilities
pi /= pi.sum()
M /= M.sum(axis=1, keepdims=True)

first letter distribution:

In [12]:
pi_initial_count

array([13180.,  6167.,   532.,   549.,  8730., 13784.,   634.,  1073.,
         251.,  5588.,  3434.,  6868.,  5387.,  1443.,   292.,  3988.,
        1022.,   515.,   793.,   857.,  2739.,  2138.])

i.e. the letter 'א' appears 13180 times at the beginning of a word, the letter 'ב' appears 6167 times and so on. 

The probabilties vector of first letters in word:

In [13]:
pi

array([0.16482417, 0.0771222 , 0.00665299, 0.00686559, 0.10917413,
       0.17237757, 0.00792857, 0.01341854, 0.00313891, 0.06988145,
       0.04294432, 0.08588865, 0.06736782, 0.01804562, 0.00365164,
       0.04987244, 0.01278075, 0.0064404 , 0.00991696, 0.01071732,
       0.03425291, 0.02673703])

sum of probabilities equals 1.

In [14]:
int(pi.sum())

1

The probabilties matrix of bi-gram where the first letter is: א

In [15]:
M[0]

array([8.50051003e-05, 4.13974838e-02, 6.80040802e-04, 2.27813669e-02,
       4.08449507e-02, 4.72203332e-02, 4.37776267e-03, 4.66678001e-02,
       1.70010201e-04, 4.56052363e-02, 2.50765046e-02, 1.73835430e-01,
       8.81927916e-02, 2.96242775e-02, 4.84529072e-03, 2.33764026e-03,
       8.37300238e-03, 2.42264536e-03, 1.36008160e-03, 6.20537232e-02,
       1.16456987e-01, 2.35591635e-01])

i.e the probability that 'א' comes after 'א' is 8.50051003e-05, the probability that 'ב' comes after 'א' is 4.13974838e-02 and so on.

sum of probabilities of every letter equals 1.

In [16]:
M.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.])

Check if spaces affect the probability:

In [17]:
get_sequence_prob('אבגד הוז חטיכ')

-32.62795532970536

In [18]:
get_sequence_prob('אבג דהו זחטיכ')

-37.72760701347405

In [19]:
torah_skips_2_s0 = ''
for i in range(0, len(torah_text_no_spaces), 2):
  torah_skips_2_s0 += torah_text_no_spaces[i]

In [20]:
torah_skips_2_s1 = ''
for i in range(1, len(torah_text_no_spaces), 2):
  torah_skips_2_s1 += torah_text_no_spaces[i]

In [21]:
torah_skips_2_s0[:100]

'באיבאליאהממאהרואציההוהושעפיהמרחלימחתלנהיוארליייווהאריאליאהוכטבידאהמיהוויהשוקאלילוימלשקאיהיירוהברואדי'

In [22]:
torah_skips_2_s1[:100]

'רשתראהמתשיותאצהרהתתובוחכלנתווואהמרפעפיממימאהמהארייווראהמתאריוובלליבנארבנחכיראהמארווחכרללוהעבייקימחוא'

In [23]:
words_probs_10 = {}

for i in range(len(torah_skips_2_s0) - 10):
  word = torah_skips_2_s0[i:i + 10]
  if get_sequence_prob(word) != float('-inf'):
    words_probs_10[word] = [get_sequence_prob(word), 'torah_skips_2_s0', i*2, i*2+10]

words_probs_10 = dict(sorted(words_probs_10.items(), key=lambda item: item[1], reverse=True))
print(len(words_probs_10))

148003


In [24]:
for i in range(len(torah_skips_2_s1) - 10):
  word = torah_skips_2_s1[i:i + 10]
  if get_sequence_prob(word) != float('-inf'):
    words_probs_10[word] = [get_sequence_prob(word), 'torah_skips_2_s1', i*2+1, i*2+11]

words_probs_10 = dict(sorted(words_probs_10.items(), key=lambda item: item[1], reverse=True))
words_probs_10

{'וימהוהותימ': [-19.35016073586796, 'torah_skips_2_s1', 86927, 86937],
 'ויהיהאלויה': [-19.61463533113735, 'torah_skips_2_s0', 157550, 157560],
 'ויהוהמלהוא': [-19.795910867160075, 'torah_skips_2_s1', 173289, 173299],
 'הויהיהאלוי': [-19.90011711111929, 'torah_skips_2_s0', 157548, 157558],
 'כלהאלכלהוא': [-20.108641137730658, 'torah_skips_2_s0', 54752, 54762],
 'אתמהוהאשרב': [-20.346834035777487, 'torah_skips_2_s1', 288207, 288217],
 'וימיואמהוי': [-20.508117577987633, 'torah_skips_2_s1', 214881, 214891],
 'ותוהנימתוי': [-20.53264518744649, 'torah_skips_2_s0', 216220, 216230],
 'והניהעימרא': [-20.53998821176989, 'torah_skips_2_s1', 216297, 216307],
 'וימתהוהימה': [-20.57601406996584, 'torah_skips_2_s0', 23188, 23198],
 'ויאתוישיני': [-20.63997889810845, 'torah_skips_2_s1', 23129, 23139],
 'הואהאשרימר': [-20.708308266785515, 'torah_skips_2_s0', 250970, 250980],
 'ברביוהוימש': [-20.76127436468196, 'torah_skips_2_s1', 161741, 161751],
 'יהיהאלויהי': [-20.798552066793178, 'torah_skips_2_s0

In [25]:
print(len(words_probs_10))

291390


In [26]:
df_10 = pd.DataFrame.from_dict(words_probs_10, orient='index').reset_index()

In [27]:
df_10.head()

,index,0,1,2,3
0,וימהוהותימ,-19.350161,torah_skips_2_s1,86927,86937
1,ויהיהאלויה,-19.614635,torah_skips_2_s0,157550,157560
2,ויהוהמלהוא,-19.795911,torah_skips_2_s1,173289,173299
3,הויהיהאלוי,-19.900117,torah_skips_2_s0,157548,157558
4,כלהאלכלהוא,-20.108641,torah_skips_2_s0,54752,54762


In [28]:
df_10.columns = ['string', 'log_prob', 'skip_start_from', 'i_start', 'i_end']
df_10['skip_start_from'] = df_10['skip_start_from'].map({'torah_skips_2_s0': 's0', 'torah_skips_2_s1': 's1'})
df_10

,string,log_prob,skip_start_from,i_start,i_end
0,וימהוהותימ,-19.350161,s1,86927,86937
1,ויהיהאלויה,-19.614635,s0,157550,157560
2,ויהוהמלהוא,-19.795911,s1,173289,173299
3,הויהיהאלוי,-19.900117,s0,157548,157558
4,כלהאלכלהוא,-20.108641,s0,54752,54762
...,...,...,...,...,...
291385,שחאצשעעיעג,-55.623760,s1,48341,48351
291386,סאאאאשאיוט,-55.778467,s0,77534,77544
291387,דאאצשעבזיז,-56.181992,s1,457,467
291388,ההעחאושחאא,-56.285469,s1,147061,147071


In [29]:
from google.colab import files

df_10.to_csv('df_10.csv', index=False ,encoding = 'utf-8-sig')
files.download('df_10.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
seq_probs_4_3_4 = {}

for i in range(len(torah_skips_2_s0) - 11):
  seq = torah_skips_2_s0[i:i + 4] + ' ' + torah_skips_2_s0[i + 4:i + 7] + ' ' + torah_skips_2_s0[i + 7:i + 11]
  if get_sequence_prob(seq) != float('-inf'):
    seq_probs_4_3_4[seq] = [get_sequence_prob(seq), 'torah_skips_2_s0', i*2, i*2+11]

seq_probs_4_3_4 = dict(sorted(seq_probs_4_3_4.items(), key=lambda item: item[1], reverse=True))
print(len(seq_probs_4_3_4))
#seq_probs_4_3_4

148798


In [31]:
for i in range(len(torah_skips_2_s1) - 11):
  seq = torah_skips_2_s1[i:i + 4] + ' ' + torah_skips_2_s1[i + 4:i + 7] + ' ' + torah_skips_2_s1[i + 7:i + 11]
  if get_sequence_prob(seq) != float('-inf'):
    seq_probs_4_3_4[seq] = [get_sequence_prob(seq), 'torah_skips_2_s0', i*2+1, i*2+12]

seq_probs_4_3_4 = dict(sorted(seq_probs_4_3_4.items(), key=lambda item: item[1], reverse=True))
print(len(seq_probs_4_3_4))
seq_probs_4_3_4

293823


{'וימי ואמ הוימ': [-20.9157127483038, 'torah_skips_2_s0', 214881, 214892],
 'ויהי האל ויהי': [-21.007206557510173, 'torah_skips_2_s0', 157550, 157561],
 'ליהמ ויה הויה': [-21.26777718906711, 'torah_skips_2_s0', 157534, 157545],
 'וימר וני לוימ': [-21.325388615844457, 'torah_skips_2_s0', 55127, 55138],
 'יוימ הוה ותימ': [-22.05955149040956, 'torah_skips_2_s0', 86925, 86936],
 'לכימ ויה אמשמ': [-22.219309008751463, 'torah_skips_2_s0', 97982, 97993],
 'אתוי והו לקדב': [-22.23787219598838, 'torah_skips_2_s0', 188306, 188317],
 'אויה והמ להוא': [-22.285387012860557, 'torah_skips_2_s0', 173287, 173298],
 'והמר הימ אמהו': [-22.371823534781914, 'torah_skips_2_s0', 126705, 126716],
 'המשר וימ ולעל': [-22.386039344551772, 'torah_skips_2_s0', 84070, 84081],
 'ויהו המל הואש': [-22.536678497679958, 'torah_skips_2_s0', 173289, 173300],
 'בימי ואמ הנימ': [-22.60001319567042, 'torah_skips_2_s0', 101355, 101366],
 'אלתמ אתע ואתו': [-22.673520023170468, 'torah_skips_2_s0', 260354, 260365],
 'והוי ויא אר

In [32]:
df_4_3_4 = pd.DataFrame.from_dict(seq_probs_4_3_4, orient='index').reset_index()

In [33]:
df_4_3_4.columns = ['string', 'log_prob', 'skip_start_from', 'i_start', 'i_end']
df_4_3_4['skip_start_from'] = df_4_3_4['skip_start_from'].map({'torah_skips_2_s0': 's0', 'torah_skips_2_s1': 's1'})
df_4_3_4

,string,log_prob,skip_start_from,i_start,i_end
0,וימי ואמ הוימ,-20.915713,s0,214881,214892
1,ויהי האל ויהי,-21.007207,s0,157550,157561
2,ליהמ ויה הויה,-21.267777,s0,157534,157545
3,וימר וני לוימ,-21.325389,s0,55127,55138
4,יוימ הוה ותימ,-22.059551,s0,86925,86936
...,...,...,...,...,...
293818,דאפמ שענ פעחא,-58.471181,s0,129369,129380
293819,חאצש עעי עגתע,-58.635491,s0,48343,48354
293820,אאלו דאה זחאא,-59.281813,s0,122904,122915
293821,שצשצ לאצ קהאא,-60.352882,s0,157197,157208


In [34]:
from google.colab import files

df_4_3_4.to_csv('df_4_3_4.csv', index=False ,encoding = 'utf-8-sig')
files.download('df_4_3_4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
seq_probs_3_3_5 = {}

for i in range(len(torah_skips_2_s0) - 11):
  seq = torah_skips_2_s0[i:i + 3] + ' ' + torah_skips_2_s0[i + 3:i + 6] + ' ' + torah_skips_2_s0[i + 6:i + 11]
  if get_sequence_prob(seq) != float('-inf'):
    seq_probs_3_3_5[seq] = [get_sequence_prob(seq), 'torah_skips_2_s0', i*2, i*2+11]

seq_probs_3_3_5 = dict(sorted(seq_probs_3_3_5.items(), key=lambda item: item[1], reverse=True))
print(len(seq_probs_3_3_5))
#seq_probs_3_3_5

148798


In [36]:
for i in range(len(torah_skips_2_s1) - 11):
  seq = torah_skips_2_s1[i:i + 3] + ' ' + torah_skips_2_s1[i + 3:i + 6] + ' ' + torah_skips_2_s1[i + 6:i + 11]
  if get_sequence_prob(seq) != float('-inf'):
    seq_probs_3_3_5[seq] = [get_sequence_prob(seq), 'torah_skips_2_s0', i*2+1, i*2+12]

seq_probs_3_3_5 = dict(sorted(seq_probs_3_3_5.items(), key=lambda item: item[1], reverse=True))
print(len(seq_probs_3_3_5))
seq_probs_3_3_5

293823


{'וימ הוה ותימי': [-21.15380859213986, 'torah_skips_2_s0', 86927, 86938],
 'ויה הוי היהאל': [-21.228722896607422, 'torah_skips_2_s0', 157542, 157553],
 'ויה והמ להואש': [-21.337881632966806, 'torah_skips_2_s0', 173289, 173300],
 'יהמ ויה הויהי': [-21.377736435448632, 'torah_skips_2_s0', 157536, 157547],
 'הוי היה אלויה': [-21.392342534630735, 'torah_skips_2_s0', 157548, 157559],
 'והו ואר הניהמ': [-21.96231900676115, 'torah_skips_2_s0', 204108, 204119],
 'אתמ הוה אשרבה': [-22.133804691050727, 'torah_skips_2_s0', 288207, 288218],
 'אלא וימ הימהש': [-22.2420804997386, 'torah_skips_2_s0', 96543, 96554],
 'וימ יוא מהוימ': [-22.71239400395598, 'torah_skips_2_s0', 214881, 214892],
 'אלו לימ והנוא': [-22.779331552376988, 'torah_skips_2_s0', 80511, 80522],
 'יהי ויה להיאל': [-22.824933398101038, 'torah_skips_2_s0', 31546, 31557],
 'ותמ הוי אשמהי': [-22.825321271582773, 'torah_skips_2_s0', 186562, 186573],
 'ישו ואת הוהימ': [-22.82860523748467, 'torah_skips_2_s0', 17534, 17545],
 'ארא איה יהוימ

In [37]:
df_3_3_5 = pd.DataFrame.from_dict(seq_probs_3_3_5, orient='index').reset_index()

In [38]:
df_3_3_5.columns = ['string', 'log_prob', 'skip_start_from', 'i_start', 'i_end']
df_3_3_5['skip_start_from'] = df_3_3_5['skip_start_from'].map({'torah_skips_2_s0': 's0', 'torah_skips_2_s1': 's1'})
df_3_3_5

,string,log_prob,skip_start_from,i_start,i_end
0,וימ הוה ותימי,-21.153809,s0,86927,86938
1,ויה הוי היהאל,-21.228723,s0,157542,157553
2,ויה והמ להואש,-21.337882,s0,173289,173300
3,יהמ ויה הויהי,-21.377736,s0,157536,157547
4,הוי היה אלויה,-21.392343,s0,157548,157559
...,...,...,...,...,...
293818,כשצ אאא רמארל,-57.656298,s0,26392,26403
293819,דאא רשג דאהחח,-58.153371,s0,162999,163010
293820,זחי חאע לחאאר,-58.851950,s0,153325,153336
293821,דאא צשע בזיזע,-60.408593,s0,457,468


In [39]:
from google.colab import files

df_3_3_5.to_csv('df_3_3_5.csv', index=False ,encoding = 'utf-8-sig')
files.download('df_3_3_5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>